In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier

# Initialiser SparkSession
sc = SparkSession.builder.appName('RandomForestClassifier').getOrCreate()

consumption =pd.read_csv("InputTrain.csv")
washing_machine = pd.read_csv("StepTwo_LabelTrain_WashingMachine.csv")
dishwasher = pd.read_csv("StepTwo_LabelTrain_Dishwasher.csv")
tumble_dryer = pd.read_csv("StepTwo_LabelTrain_TumbleDryer.csv")
microwave = pd.read_csv("StepTwo_LabelTrain_Microwave.csv")
kettle = pd.read_csv("StepTwo_LabelTrain_Kettle.csv")

def transform(machine,name):
    time_stamps = []
    for elt in consumption.columns :
        if str.startswith(elt,'TimeStep') :
            time_stamps.append(elt)
    machine = pd.melt(machine,id_vars=['Index','House_id'],value_vars= time_stamps, var_name='time',value_name = name)

    machine = pd.DataFrame(machine)
    times = []
    for elt in machine['time']:
        times.append(int(elt[9:]))
    machine['time'] = times
    machine = machine.sort_values(['Index','time'])
    return machine

23/04/10 15:48:55 WARN Utils: Your hostname, BillyTheChild resolves to a loopback address: 127.0.1.1; using 172.21.53.139 instead (on interface eth0)
23/04/10 15:48:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/04/10 15:48:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
washing_machine = transform(washing_machine,'washing_machine')
dishwasher = transform(dishwasher,'dishwasher')
tumble_dryer = transform(tumble_dryer,'tumble_dryer')
microwave = transform(microwave,'microwave')
kettle = transform(kettle,'kettle')

machines = [washing_machine,dishwasher,tumble_dryer,microwave,kettle]

consumption = transform(consumption,'watt')
display(consumption)

for elt in machines:
    display(elt)

,Index,House_id,time,watt
0,0,1,0,180.0
10421,0,1,1,180.0
20842,0,1,2,180.0
31263,0,1,3,181.0
41684,0,1,4,180.0
...,...,...,...,...
22467675,10420,16,2155,366.5
22478096,10420,16,2156,376.0
22488517,10420,16,2157,367.0
22498938,10420,16,2158,353.0


,Index,House_id,time,washing_machine
0,0,1,0,0.0
10421,0,1,1,0.0
20842,0,1,2,0.0
31263,0,1,3,0.0
41684,0,1,4,0.0
...,...,...,...,...
22467675,10420,16,2155,0.0
22478096,10420,16,2156,0.0
22488517,10420,16,2157,0.0
22498938,10420,16,2158,0.0


,Index,House_id,time,dishwasher
0,0,1,0,0.0
10421,0,1,1,0.0
20842,0,1,2,0.0
31263,0,1,3,0.0
41684,0,1,4,0.0
...,...,...,...,...
22467675,10420,16,2155,0.0
22478096,10420,16,2156,0.0
22488517,10420,16,2157,0.0
22498938,10420,16,2158,0.0


,Index,House_id,time,tumble_dryer
0,0,1,0,0.0
10421,0,1,1,0.0
20842,0,1,2,0.0
31263,0,1,3,0.0
41684,0,1,4,0.0
...,...,...,...,...
22467675,10420,16,2155,0.0
22478096,10420,16,2156,0.0
22488517,10420,16,2157,0.0
22498938,10420,16,2158,0.0


,Index,House_id,time,microwave
0,0,1,0,0.0
10421,0,1,1,0.0
20842,0,1,2,0.0
31263,0,1,3,0.0
41684,0,1,4,0.0
...,...,...,...,...
22467675,10420,16,2155,0.0
22478096,10420,16,2156,0.0
22488517,10420,16,2157,0.0
22498938,10420,16,2158,0.0


,Index,House_id,time,kettle
0,0,1,0,0.0
10421,0,1,1,0.0
20842,0,1,2,0.0
31263,0,1,3,0.0
41684,0,1,4,0.0
...,...,...,...,...
22467675,10420,16,2155,0.0
22478096,10420,16,2156,0.0
22488517,10420,16,2157,0.0
22498938,10420,16,2158,0.0


In [11]:
data = pd.concat([consumption,washing_machine['washing_machine'],dishwasher['dishwasher'],tumble_dryer['tumble_dryer'],microwave['microwave'],kettle['kettle']],axis=1,join="inner")
display(data)

,Index,House_id,time,watt,washing_machine,dishwasher,tumble_dryer,microwave,kettle
0,0,1,0,180.0,0.0,0.0,0.0,0.0,0.0
10421,0,1,1,180.0,0.0,0.0,0.0,0.0,0.0
20842,0,1,2,180.0,0.0,0.0,0.0,0.0,0.0
31263,0,1,3,181.0,0.0,0.0,0.0,0.0,0.0
41684,0,1,4,180.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
22467675,10420,16,2155,366.5,0.0,0.0,0.0,0.0,0.0
22478096,10420,16,2156,376.0,0.0,0.0,0.0,0.0,0.0
22488517,10420,16,2157,367.0,0.0,0.0,0.0,0.0,0.0
22498938,10420,16,2158,353.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Sélectionner les colonnes de la variable cible et de la variable indépendante
model_df = assembled_df.select(['appliance1', 'features'])

# Diviser les données en ensembles de formation et de test
train_data, test_data = model_df.randomSplit([0.7, 0.3])




NameError: name 'assembled_df' is not defined

In [ ]:
# Initialiser le modèle de classification Random Forest
rf = RandomForestClassifier(numTrees=10)

# Entraîner le modèle sur les données de formation
model = rf.fit(train_data)

In [ ]:
# Faire des prédictions sur les données de test
predictions = model.transform(test_data)

# Évaluer les performances du modèle en utilisant l'évaluateur de classification binaire
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='appliance1')
auc = evaluator.evaluate(predictions)

# Afficher l'AUC du modèle
print("AUC:", auc)
